# Introduction to B cell repertoire analysis using the Immcantation framework

![](assets/cover.png)

The field of high-throughput adaptive immune receptor repertoire sequencing (AIRR-seq) has experienced significant growth in recent years, but this growth has come with considerable complexity and variety in experimental design. These complexities, combined with the high germline and somatic diversity of immunoglobulin repertoires, present analytical challenges requiring specialized methodologies.

<br><strong>This tutorial covers:</strong>
<ul><li>V(D)J gene annotation and novel polymorphism detection
<li>Clonotype assignment
<li>Diversity analysis
<li>Mutational load profiling
<li>Modeling of somatic hypermutation targeting
<li>Quantification of selection pressure
</ul>

![](assets/workflow_h_cmd.png)


# Resources

- Questions, issues, help immcantation@googlegroups.com

- Documentation http://immcantation.org

- Source code and bug reports https://bitbucket.org/kleinstein/

- Docker/Singularity container https://hub.docker.com/r/kleinstein/immcantation

- Slides and example data https://goo.gl/FpW3Sc


# Inside this container

This container comes with software, scripts, reference germlines and example data ready to use. The commands `versions report` and `builds report` show the versions and/or dates of the tools and data.

## Software versions
Use this command to list the software versions

In [ ]:
%%bash
versions report

## Builds versions
Use this command to list the date and changesets used during the image build.

In [ ]:
%%bash
builds report

## Example data

`../data/input.fasta` Processed reads from one healthy donor (PGP1) 3 weeks after flu vaccination (Laserson et al. (2014))

## Reference germlines

The container has human and mouse reference germlines from IMGT, and the corresponding IgBLAST databases.

In [ ]:
%%bash
ls /usr/local/share/germlines/imgt

In [ ]:
%%bash
ls /usr/local/share/igblast

# V(D)J segment annotation

First step in the analysis of processed reads (`input.fasta`) is to obtain reads annotated with V(D)J alleles and CDR3 sequences.

Tools: IMGT/HighV-QUEST, IgBLAST, or other. Change-O provides a wrapper script (`AssignGenes.py`) to run IgBLAST using the reference germlines in the container.

##  A test with 200 sequences

For a quick test of Change-O's V(D)J assignment tool, use `SplitSeq.py` to extract 200 sequences from `input.fasta` and then assing genes with `AssignGenes.py`.

In [ ]:
%%bash
# reference germlines in /usr/local/share/igblast
mkdir -p results/igblast
SplitSeq.py sample -n 200 --outdir results --fasta -s ../data/input.fasta


`AssignGenes.py` performs V(D)J assignment with IgBLAST. It requires the input sequences (`-s`) and a reference germlines database (`-b`), in this case the one already available in the container. Specify the organism with `--organism` and the type of receptor with `--loci` (ig for B cell receptor). Use `--format blast` to specify the results should be in the `fmt7` format. Specify the number of processors with `--nproc`. 

In [ ]:
%%bash
AssignGenes.py igblast -s results/input_sample1-n200.fasta \
-b /usr/local/share/igblast --organism human --loci ig \
--format blast --outdir results/igblast --nproc 8

## With all data

In the same command used above, just change the input file (`-s`) to use all data. This will take some time to finish... 

In [ ]:
%%bash
AssignGenes.py igblast -s ../data/input.fasta \
-b /usr/local/share/igblast --organism human --loci ig \
--format blast --outdir results/igblast --nproc 8

# Changeo-O: Data standardization

Gupta et al. (2015)

Once the V(D)J annotation is finished, parse IgBLAST results into a standardized format suitable for downstream analysis. All tools in the framework use this format, which allows for interoperability, and provides flexibility to design complex workflows.

In this example analysis, parse the `fmt7` results from IgBLAST into the [Change-O](https://changeo.readthedocs.io) format: a tabulated text file with rows being the sequences and columns their annotations, using standard column names as described here https://changeo.readthedocs.io/en/latest/standard.html.

The command line tool `MakeDb.py igblast` needs the original input fasta (`-s`) and the aligner results (`-i`). The argument `--format changeo` specifies to use the Change-O format. `-r` contains the path to the reference germlines retrieved from IMGT and already available in the container. `--regions` specifies IMGT FWR and CDRs should be included in the output. `--scores` specifies alignment score metrics should be included in the output.

In [ ]:
%%bash
mkdir -p results/changeo
MakeDb.py igblast \
-s ../data/input.fasta -i results/igblast/input_igblast.fmt7 \
--format changeo \
-r /usr/local/share/germlines/imgt/human/vdj/ --outdir results/changeo \
--regions --scores --outname data

## Subset to productive heavy chain sequences

To focus the analysis on productive sequences from the heavy chain, data from the previous step needs to be filtered.

### Select functional sequences

`ParseDb.py select` finds the rows in the file `-d` for which the column `FUNCTIONAL` (specified with `-f`) contains the values `T` or `TRUE` (specified by `-u`). The prefix `data_f` will be used in the name of the output file (specified by `--outname`).

In [ ]:
%%bash
ParseDb.py select -d results/changeo/data_db-pass.tab \
-f FUNCTIONAL -u T TRUE --outname data_f

### Select heavy chain sequences

Next, `ParseDb.py select` finds the rows in the file `-d` from the previous step for which the column `V_CALL` (specified with `-f`) contains (pattern matching specified by `--regex`) the word `IGHV` (specified by `-u`). The prefix `data_fh` (standing for functional and heavy) will be used in the name of the output file (specified by `--outname`).

In [ ]:
%%bash
ParseDb.py select -d results/changeo/data_f_parse-select.tab \
-f V_CALL -u IGHV --regex --outname data_fh

# Some set up for the notebook to run R code
The next two lines of code are required for this Jupyter Notebook to use the `R magic` and run `R` code. Next steps in the tutororial will use the R packages in the Immcantation framework.

In [ ]:
import rpy2.rinterface
%load_ext rpy2.ipython

# TIgGER: genotyping, including novel V gene alleles

Gadala-Maria et al. (2015) Gadala-Maria et al. (2019)

V(D)J assignment is a key step in analyzing repertoires, and it is done by matching the sequences against a database of known V(D)J alleles. But databases are incomplete, and this process will fail for sequences that utilize previously undetected alleles. Also, there will be situations where the aligner fings a tie and can't decide between multiple possible calls, and therefore some sequences have multiple annotations. The TIgGER R package can help here by <strong>finding novel alleles</strong> as well as <strong>inferring the individual’s genotype to correct calls</strong>. 


After loading the libraries needed, `readChangeoDb` is used to read the tabulated data generated in the previous step.

In [ ]:
%%R
suppressPackageStartupMessages(library(alakazam))
suppressPackageStartupMessages(library(tigger))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggplot2))
dir.create("results/tigger")
db <- readChangeoDb("results/changeo/data_fh_parse-select.tab")
# Show column names
colnames(db)

`readIgFasta` loads the V germline reference alleles that were used in the V(D)J assignment, in this case the sequences in the Immcantation container.

In [ ]:
%%R
ighv <- readIgFasta("/usr/local/share/germlines/imgt/human/vdj/imgt_human_IGHV.fasta")
# See first germline
ighv[1]

## Identify polymorphisms

Once data are loaded, for each allele, `findNovelAlleles` analyzes the sequences that were assigned the allele, and evaluates the mutation frequency at each position as a function of sequence-wide mutation counts. We expect polymorphic positions to exhibit a high mutation frequency despite sequence-wide mutation count. In this example, TIgGER finds one novel allele.

In [ ]:
%%R
# find novel alleles
nv <- findNovelAlleles(db, germline_db=ighv, nproc=8)

In [ ]:
%%R
# show novel alleles
selectNovel(nv)

The function `plotNovel` helps visualize the supporting evidence for calling the novel allele. The mutation frequency of the novel polymorphism is highlighted in red. The position is consistently mutated, it is high for all sequence-wide mutation count (top panel). By the default, `findNovelAlleles` uses several thresholds to avoid false positives. For instance, to avoid calling novel alleles resulting from clonally-related sequences, TIgGER requires that sequences perfectly matching the potential novel allele utilize a wide range of combinations of J gene and junction length. The novel allele is expected to be found in diverse J genes and juntion length combinations and this can be observed in the bottom panel.



In [ ]:
%%R
# visualize
plotNovel(db, selectNovel(nv)[1,])

## Genotyping, including novel V gene alleles

The next step is to infer the subject’s genotype and correct gene calls. The idea is to remove calls that were assigned to a sequence, but that after looking at the whole individual’s repertoire, they are relatively rare and therefore unlikely to be correct allele calls in the context of the particular individual.

`inferGenotype` uses a frequency method to determine the genotype. It finds the minimum number set of alleles that can explain the majority of each gene’s calls. The most common allele of each gene is included in the genotype first, then the next most common allele is added until the desired fraction of alleles can be explained.

In [ ]:
%%R
gt <- inferGenotype(db, germline_db=ighv, novel=nv)
# save genotype inf .fasta format to be used later with CreateGermlines.py
gtseq <- genotypeFasta(gt, ighv, nv)
writeFasta(gtseq, "results/tigger/v_genotype.fasta")
# show first 3 rows
gt %>% arrange(TOTAL) %>% slice(1:3)

The individual's genotype can be visualized with `plotGenotype`. Each row is a gene and has as many colored cells as alleles for that gene in the genotype.

In [ ]:
%%R
plotGenotype(gt)## Genotyping, including novel V gene alleles

## Reassign alleles

The gene calls can be corrected with `reassignAlleles`. Mistaken allele calls can come from sequences which by chance have been mutated to look like another allele, and after learning the individual's genotype, they can be corrected. `reassignAlleles` saves the corrected calls in a new column, `V_CALL_GENOTYPED`. With `writeChangeoDb` the updated data is saved as a tabulated file for use in next steps.


In [ ]:
%%R
db <- reassignAlleles(db, gtseq)
# show some of the corrected gene calls
db %>% filter(V_CALL != V_CALL_GENOTYPED) %>%
sample_n(3) %>% select(V_CALL, V_CALL_GENOTYPED)

In [ ]:
%%R
writeChangeoDb(db, "results/tigger/data_fh_genotyped.tab")

# Clonal diversity analysis

Goal: Partition (cluster) sequences into clonally related lineages. Each lineage is a group of sequences that came from the same original naive cell.

Summary of the key steps:
    
- Determine clonal clustering threshold: sequences which are under this cut-off are clonally related.

- Assign clonal groups: add and annotation (columname `CLONE`) that can be uses to identify a group of sequences that came from the same original naive cell.

- Reconstruct germline sequences: figure out the germline sequence of the common ancestor, before mutations are introduced during clonal expansion and SMH.

- Analize clonal diversity: number and size of clones? any expanded clones?


![](assets/clonalexpansion.png)



## Clonal assignment using the CDR3 as a fingerprint

Gupta et al. (2017) Nouri and Steven H Kleinstein (2018b)

Clonal relationships can be inferred from sequencing data. Hierarchical clustering is the most widely used method for identify clonally related sequences. This method requires a measure of distance between pairs of sequences and a choice of linkage to define the distance between groups of sequences. And because the result will be a tree, a threshold to cut the hierachy into clonal groups is also needed. 

The figure bellow-left shows a tree and the chosen threshold (red dotted line). Sequences which have a distance under this cut-off are considered clonally whereas sequences which have distance above this cut-off, are considered too different and therefore not clonally related. `shazam` provides methods to calculate the distance between sequences and find a threshold (`distToNearest` and `findThreshold`). Analyzing the distance to the nearest neighbour is enough to distiguish clonal relationships. The figure below-right shows the distribution of the distances to the nearest neighbour for a repertoire. Typically, the distribution is bimodal. The first mode (left) represents sequences which are clonally related, and the second mode is representative of sequences which are not clonally related. The threshold can be seen, here, by eye, at the equal-distance between two modes. However, `findThreshold` can be used to automatically find the threshold.

<img src="assets/hclust.png" width="35%" align="left"><img src="assets/distNearest.png" width="30%" align="left">

## SHazaM: Tuning of the clonal assignment threshold

Gupta et al. (2015)

The repertoire can be large and it is common to first split data into smaller groups of sequences using some similarity requirement (same V gene, same J gene and same junction length)  and on each group, apply the hierarchical clustering on the junction sequence. `distToNearest` performs this grouping step then, for each group, it counts the mismatches in the junction region between all the sequences, and for each sequence, returns the smallest non-zero value. At the end of this step, `db` has been added a new column, `DIST_NEAREST`, which contains the distances to the most similar sequence in the group.

`findThreshold` uses the distribution of distances calculated in the previous step and either a `density` or `mixture` method to determine a threshold that can be user to cut the hierarchy. The function `plot` can be used to visualize the distance distribution and the threshold. Sequences with less missmatches are expected to be clonally related and contribute to the first mode in the distribution plot, while those sequences with higher number of mismatches are not expected to be clonally related and therefore will appear on the second mode, in farther distances.

In [ ]:
%%R -o thr
# Get the distance to nearest neighbors
suppressPackageStartupMessages(library(shazam))
db <- distToNearest(db, model="ham", normalize="len", vCallColumn="V_CALL_GENOTYPED", nproc=4)
# Determine threshold
threshold <- findThreshold(db$DIST_NEAREST, method="density")
thr <- round(threshold@threshold, 2)
thr

In [ ]:
%%R
# plot the distribution
plot(threshold)

### New method: Spectral clonal clustering methods (SCOPer)

Nouri and Steven H Kleinstein (2018a)

The bimodal distribution is not always detectable and therefore `findThreshold` may fail to determine the cut-off. We have developed an unsupervised computational framework to perform the cloning based on the local  relationships of the sequences among different neighborhoods. This can be done by `defineClonesScoper()` function from the SCOPer R package.



## Change-O: Clonal assignment

Once a threshold is decided, the command line tool `DefineClones.py` from Change-O performs the clonal assignment. There are some arguments that need to be specified. 
- `--model`: the distance metric used in `distToNearest`
- `--norm`: type of normalization used in `distToNearest`
- `--dist`: the cut-off from `findThreshold`

At the end of this step, the output file will have an additional column, `CLONE`, with the clone identifiers which have been assigned to each sequence.

In [ ]:
# Assign clonal groups
! DefineClones.py -d results/tigger/data_fh_genotyped.tab --vf V_CALL_GENOTYPED \
--model ham --norm len --dist {thr[0]} --nproc 8 \
--outname data_fh_genotyped --outdir results/changeo/

## Change-O: add full length germline sequences

Next step is to identify the V(D)J germline sequences from which the observed sequences originated. These germlines will be used in downstream analysis to infer mutations and reconstruct lineages.

`CreatGermlines.py` takes the alignment information in the Change-O file (`-d`) as well as the reference database used by the alignment software (`-r`) and generates a full-length germline sequence for each individual observed sequence, unless the argument `--cloned` is used, in which case, the function assign the same germline to all sequences belonging to the same clone. `-g` is used to specify the type of germlines to be reconstructed. In this example, `-g dmask` is used to mask the D region, that is, to replace nucleotides in the  N/P and D-segments with N's. This is done because the D-segment call for B cell receptor alignments is often low confidence. At the end of this step, the data file will have the germline sequence in the `GERMLINE_IMGT_D_MASK`.




In [ ]:
%%bash
CreateGermlines.py -d results/changeo/data_fh_genotyped_clone-pass.tab \
-r /usr/local/share/germlines/imgt/human/vdj/*IGH[DJ].fasta results/tigger/v_genotype.fasta \
-g dmask --cloned --vf V_CALL_GENOTYPED \
--outname data_fh_genotyped

## Alakazam: Analysis of clonal diversity

Gupta et al. (2015)

Once the original database is annotated with clonal relatioships (`CLONE`), the R package `alakazam` is used to analyze clonal diversity and abundance.

### Load data and subset to IGHM, IGHG and IGHA

The tab-delimited file is loaded with the function `readChangeoDb`. In this example, the analysis will focus on sequences that have been annotated isotypes IGHM, IGHG or IGHA. The function `subset` is used to keep only those sequences.

In [ ]:
%%R
dir.create("results/alakazam")
# Upate names in alakazam’s default colors (IG_COLORS)
names(IG_COLORS) <- c( "IGHA", "IGHD", "IGHE", "IGHG", "IGHM","IGHK", "IGHL")
# Read file that passed cloning and germline creation
db <- readChangeoDb("results/changeo/data_fh_genotyped_germ-pass.tab")
# Subset data to IgM, IgG and IgA
db <- subset(db, ISOTYPE %in% c("IGHM","IGHG","IGHA"))

### Estimate clonal abundance

`estimageAbundance` estimates the complete clonal relative abundance distribution and confidence intervals on clone sizes using bootstrapping. In this example, the abundance is analyzed by isotype (`group=ISOTYPE`). In the figure below, generated with the function `plot`, the y-axis shows the clone size as a percent of the repertoire and the x-axis is a rank of clone sizes, where clones are sorted by size from larger (rank 1, left) to smaller (right). The shaded areas are confidence intervals. 



In [ ]:
%%R
# Calculate rank-abundance curve
a <- estimateAbundance(db, group="ISOTYPE")
plot(a, colors=IG_COLORS)

### Clonal diversity

Another common way to approach clonal abundance analysis is the diversity statistics. Diversity scores (`D`) are calculated using the generalized diversity index (Hill numbers), which is a generalized case which covers different measures of diversity in a single function over a single varying parameter `q`.

The function `rarefyDiversity` resamples the sequences and calculates diversity scores (`D`) over an interval of diversity orders (`q`) and the result can be visualized with `plot`. The diversity (`D`) is shown on the y-axis and the x-axis is Hill numbers over parameter `q`. `q=0` corresponds to Species Richness, `q=1` corresponds Shanon Entropy and `q=2` corresponds Simpson Index. This figure is useful to visualize if diversity depends on the statistics used or it is an universal. For example, you can see that the blue curve is changing when you change `q`, whereas the  diversity of the green and red curves doesn't change significantly along values of `q`. 

In [ ]:
%%R
# Generate Hill diversity curve
d <- rarefyDiversity(db, group="ISOTYPE")
p <- plot(d, silent=T)
p + 
   geom_vline(xintercept=c(0,1,2), color="grey50", linetype="dashed") + 
   geom_text(data=data.frame(q=c(0,1,2), y=round(max(p$data$D_UPPER)/2), 
             label=c("Richness", "Shannon", "Simpson")), 
             aes(x=q, y=y,label=label), size=3, angle=90, vjust=-0.4, inherit.aes = F, color="grey50")

# Alakazam: Physicochemical properties of the CDR3

The CDR3 is the most variable region of the antibody and its physicochemical properties are key contributors to the final antigen specificity. The function `aminoAcidProperties` in the `alakazam` R package can calculate several amino acid sequence physicochemical properties like length, hydrophobicity (GRAVY index), bulkiness, polarity and net charge, among others. In this example, the junction sequence (specified by `seq="JUNCTION"`), which is available in the form of a nucleotide sequence (`nt=T`), is trimmed to remove the first and last codon/amino acids before calculating the properties. Trimming of these codons, which correspond to the conserved positions that delimit the CDR3, means that the physicochemical analyzed will be those of the CDR3, and consequently naming the new columns that will be generated with `label="CDR3"` is appropriate.


The figure below, generated with standard `ggplot` commands, shows the distribution of the CDR3 amino acid length by isotype.




In [ ]:
%%R
# Calculate CDR3 amino acid properties
db <- aminoAcidProperties(db, seq="JUNCTION", nt=T, trim=T, label="CDR3")
# Plot
ggplot(db, aes(x=ISOTYPE, y=CDR3_AA_LENGTH)) + theme_bw() +
ggtitle("CDR3 length") + xlab("Isotype") + ylab("Amino acids") +
scale_fill_manual(name="Isotype", values=IG_COLORS) +
geom_boxplot(aes(fill=ISOTYPE))

# Alakazam: family, gene and allele usage

The study of biases in gene segment usage is also of interest, as it has been observed to exist when comparing different populations, developmental stages or health vs disease. The function `countGenes` from the `alakazam` R packages can determine the count and relative abundance of V(D)J alleles, genes, or families within groups.

## V family usage by isotype

Here, `countGenes` counts V gene family usage (`mode="family"`) by isotype (`groups="ISOTYPE"`) using the allele calls in the column `V_CALL_GENOTYPED`. `--clone` specifies each clone will be considered only once, using for counting the most common gene within the clone.

In [ ]:
%%R
# V family usage by isotype
# "CLONE" specifies to consider one gene per clone,
# the most common gene within each clone
usage_fam_iso <- countGenes(db, gene="V_CALL_GENOTYPED", groups="ISOTYPE", clone="CLONE", mode="family")
# groups="ISOTYPE", then usage by isotype sums 1
usage_fam_iso %>% group_by(ISOTYPE) %>% summarize(total=sum(CLONE_FREQ))

In [ ]:
%%R
ggplot(usage_fam_iso, aes(x=ISOTYPE, y=CLONE_FREQ)) + 
geom_point(aes(color=ISOTYPE), size=4) +
scale_fill_manual(name="Isotype", values=IG_COLORS) + 
facet_wrap(~GENE, nrow=1) +
theme_bw() + ggtitle("V Family usage by isotype") +
ylab("Percent of repertoire") + xlab("Isotype")


# Alakazam: lineage reconstruction


Related to the clonal diversity analysis, is lineage reconstruction. B cell repertoires often consist of hundreds to thousands of separate low-diversity lineages. A lineage recapitulates the relationships between clonally related B cells and uncovering these relationships is necessary to characterize processes that depend on affinity maturation, such as age related changes in the repertoire, the development of immunological memory or the response to vaccination. The R package `alakazam` uses PHYLIP to reconstruct lineages following a maximum parsimony technique. PHYLIP is already installed in the Immcantation container.

Before perfoming the lineage reconstruction, some preprocessing in needed. The code below shows an example of such preprocessing done for one of the largest clones. The function `makeChangeoClone` takes as input a data.frame with information for a clone (`db_clone`). `text_fields="ISOTYPE"` specifies annotation in the column `ISOTYPE` should be merged during duplicate removal. For example, if one of two duplicated sequences is annotated as IGHM and the second one is an IGHG, the collapsed sequence will have the ISOTYPE value IGHM,IGHG. The preprocessing done by `makeChangeoClone` includes mask gap positions, mask ragged ends, remove duplicated sequences and merges annotations associated with duplicated sequences.

In [ ]:
%%R
# Select one clone, the 2nd largest, just as an example
largest_clone <- countClones(db) %>% slice(2) %>% select(CLONE)
# Subset db, get db with data for largest_clone
db_clone <- subset(db, V_CALL="V_CALL_GENOTYPED", CLONE == largest_clone[['CLONE']])
# Build tree from a single clone
x <- makeChangeoClone(db_clone, text_fields="ISOTYPE")

Lineage reconstruction is done with `buildPhylipLineage`. This function uses the `dnapars` tool from PHYLIP, which in the container is located at `/usr/local/bin/dnapars`, to build the lineage, and returns an `igraph` object. This object can be quickly visualized with the command `plot`. It is possible to use `igraph` or other tools to resize nodes or add colors to help you answer your biologically relevant question.

In [ ]:
%%R
# Lineage reconstruction
g <- buildPhylipLineage(x, dnapars="/usr/local/bin/dnapars")
suppressPackageStartupMessages(library(igraph))
plot(g)

# Alakazam: lineage topology analysis

`alakazam` has several functions to analyze the reconstructed lineage. `getMRCA` retrieves the first non-root node of a lineage tree, namely Most Recent Common Ancestor (MRCA), meaning the ancestor which is most related to the germline, but not exactly the germline. `getpathlength` calculates all the path lengths from the tree root to all individual sequences. `summarizeSubtrees` calculates summary statistics for each node of a tree (node name, name of the parent node, number of edges leading from the node, total number of nodes within the subtree rooted at the node...). `tableEdges` creates a table of the total number of connections (edges) for each unique pair of annotations within a tree over all nodes.

In [ ]:
%%R
# Retrieve the most ancestral sequence
getMRCA(g, root="Germline")

In [ ]:
%%R
# Calculate distance from germline
getPathLengths(g, root="Germline") %>% top_n(2)

In [ ]:
%%R
# Calculate subtree properties
summarizeSubtrees(g, fields="ISOTYPE") %>% top_n(2)

In [ ]:
%%R
# Tabulate isotype edge relationships
tableEdges(g, "ISOTYPE", exclude=c("Germline", NA))

# SHazaM: mutational load

B cell repertoires also differ in the number of mutations introduced during somatic hypermutation (SHM). The `shazam` R package is dedicated pretty much entirely to the analysis of SHM, including SHM targeting models and selection pressure.

The most intuitive thing to start with, is simply count the mutations in a given sequence. Having identified the germline (`GERMLINE_IMGT_D_MASK`) in previous steps, somatic mutations are identified comparting the observed sequence (`SEQUENCE_IMGT`) to the germline. `observedMutations` can count mutations in either absolute counts (`frequency=F`) or as a frequency (the number of mutations divided by the number of informative positions, `frequency=T`). Replacement (R) and silent (S) mutations can be counted together (`conmbine=T`) or independelty (`combine=F`). Counting can be limited to mutatations happening in a particular region (for example, to focus on the V region `regionDefinition=IMGT_V`) or use the whole sequence (`regionDefinition=NULL`).

Standard `ggplot` commands can be used to plot boxplots to visualize the distribution of mutation counts in the column `MU_COUNT` that `observedMutations` added to `db`.

In [ ]:
%%R
# Calculate total mutation count, R and S combined
db <- observedMutations(db,
            sequenceColumn="SEQUENCE_IMGT", 
            germlineColumn="GERMLINE_IMGT_D_MASK",
            regionDefinition=NULL, frequency=F, combine = T, nproc=4)
ggplot(db, aes(x=ISOTYPE, y=MU_COUNT, fill=ISOTYPE)) +
geom_boxplot() +
scale_fill_manual(name="Isotype",values=IG_COLORS) +
xlab("Isotype") + ylab("Mutation count") + theme_bw()

# SHazaM: models of SHM targeting biases
Yaari et al. (2013)

There are known biases in SHM patterns, which are typically associated with nucleotide motifs. There are number of models included in `shazam` which have been built based on experimental datasets for human and mouse, for both light and heavy chains. In addition, `shazam` provides methods to generate the model from the input data.
![](assets/5mer.png)
`createTargetingModel` can generate models of SHM targeting biases from the input data (`db`) based on 5-mer models. That means for any given 5 nucleotides, it evaluates the chances that the base at the center of that 5-mer can be mutated, and what is it going to be mutated to. The model can be visualized as a hedgehog plot with `plotMutability`. SHM does not target all positions in the BCR equally. The known hot-spot motifs are shown in red and green, the known cold-spot motifs in blue, and the known neutral-spot motifs in grey. The bars outward are the relative mutation rates. The middle circle is the central nucleotide, here shown by a green circle on the left plot (nucleotide A) and an orange circle on the right plot (nucleotide C). Moving inward and outward of the circles covers the rest of the 5-mer nucleotides. The expectation is to see higher relative mutation rates at hot-spot motifs, in contrary to cold- and neutral-motifs.

In [ ]:
%%R
dir.create("results/shazam")
# Build and plot SHM targeting model
m <- createTargetingModel(db, vCallColumn="V_CALL_GENOTYPED")
# nucleotides: center nucleotide characters to plot
plotMutability(m, nucleotides=c("A","C"), size=1.2)

# SHazaM: quantification of selection pressure

Yaari et al. (2012)

During the immune response, B cells undergo cycles of SHM and affinity-dependent selection and accumulate mutations that improve their ability to bind antigens. The ability to estimate selection from sequence data is of interest to understand the events driving physiological and pathological immune responses. `shazam` incorporates the most up-to-date methods from the Bayesian estimation of Antigen-driven SELectIoN (BASELINe) framework, which can detect and quantify selection ($\sum$), based on the analysis of somatic mutation patterns. Briefly, `calcBaseline` looks into the frequency of replacement and silent mutations, normalized by some expectation that comes from a targeting model, and generates a probability density function (PDF), which can be used to compare selection across conditions. An increased frequency of R mutations suggests positive selection, and a decreased frequency of R mutations as points towards negative selection .
<br><br>
<div style="float:left; margin-right:2em">
    
$\sum \equiv log  \frac{\pi/(1-\pi)}{\hat{\pi}/(1-\hat{\pi})}$ 
</div>
<div>
positive Σ: Replacement frequency higher than expected

negative Σ: Replacement frequency lower than expected
</div>

Selection pressure analysis starts collapsing each clone to one representative sequence and germline with the function `collapseClones`. Individual sequences within clonal groups are not, strictly speaking, independent events and it is generally appropriate to only analyze selection pressures on an effective sequence for each clonal group, to prevent counting multiple times the mutations. Then, `calcBaseline` takes the representative clone sequences and calculates the selection pressure. The analysis can be focused on a particular region, here the V region (`regionDefinition=IMGT_V`). Because the frequency of somatic mutations is not uniform across the BCR, we may also be interested to evaluate the selection pressure for different Complementary Determining Regions (CDR) or Framework Regions (FWR), or even different isotopes or subjects. This can be done with `groupbaseline`, which convolves the PDFs into a single PDF for each individual group. 


These PDF can be visualized with `plot`. FWR are involved in the maintenance of the structure of the BCR and typically have a lower observed mutation frequency, because mutations that alter the amino acid sequence are often negatively selected. CDRs are the areas of the BCR that bind to the antigen, contain more hotspot motifs and their structure is less constrained, and usually have higher observed mutation frequencies. In the example, negative selection is observed in the FWR, and almost no selection is observed in the CDR. Sequences annotated with IGHM isotype show a more positive selection strength than sequences annotated with IGHG and IGHA, on both CDR and FWR regions.

In [ ]:
%%R
# Calculate clonal consensus and selection using the BASELINe method
z <- collapseClones(db)
b <- calcBaseline(z, regionDefinition=IMGT_V)
# Combine selection scores for all clones in each group
g <- groupBaseline(b, groupBy="ISOTYPE")
# Plot probability densities for the selection pressure
plot(g, "ISOTYPE", sigmaLimits=c(-1, 1), silent=F)

# SHazaM: Built in mutation models

`shazam` comes with serveral built in targeting models, mutation models and sequence region definitions. They are used by different functions and they can be used to change their default behaviour. For example, mutations can be considered as R when they introduce a change in the amino acid (`NULL`) or when they introduce a change in the side chain charge (`CHARGE_MUTATIONS`).

Empirical SHM Targeting Models

- HH_S5F: Human heavy chain 5-mer model

- HKL_S5F: Human light chain 5-mer model

- MK_RS5NF: Mouse light chain 5-mer model

Mutation Definitions

- NULL: Any change in amino acid is a replacement mutation.

- CHARGE_MUTATIONS:  Replacements are changes in side chain charge

- HYDOPATHY_MUTATIONS: Replacements are changes in hydrophobicity class

- POLARITY_MUTATIONS: Replacements are changes in polarity

- VOLUME_MUTATIONS: Replacements are change in volume class

Sequence Region Definitions

- NULL:  Full sequence

- IMGT_V:  V segment broken into combined CDRs and FWRs

- IMGT_V_BY_REGIONS: V segment broken into individual CDRs and FWRs


# References

Gadala-Maria,D. et al. (2019) Identification of subject-specific
immunoglobulin alleles from expressed repertoire sequencing data. Frontiers
in Immunology, 10, 129.

Gadala-Maria,D. et al. (2015) Automated analysis of high-throughput b-cell
sequencing data reveals a high frequency of novel immunoglobulin v gene
segment alleles. Proceedings of the National Academy of Sciences,
201417683.

Gupta,N.T. et al. (2017) Hierarchical clustering can identify b cell clones
with high confidence in ig repertoire sequencing data. The Journal of
Immunology, 1601850.

Gupta,N.T. et al. (2015) Change-o: A toolkit for analyzing large-scale b cell
immunoglobulin repertoire sequencing data. Bioinformatics, 31, 3356–3358.

Laserson,U. et al. (2014) High-resolution antibody dynamics of
vaccine-induced immune responses. Proc. Natl. Acad. Sci. U.S.A., 111,
4928–4933.

Nouri,N. and Kleinstein,S.H. (2018a) A spectral clustering-based method
for identifying clones from high-throughput b cell repertoire sequencing data.
Bioinformatics, 34, i341–i349.

Nouri,N. and Kleinstein,S.H. (2018b) Optimized threshold inference for
partitioning of clones from high-throughput b cell repertoire sequencing
data. Frontiers in immunology, 9.

Stern,J.N. et al. (2014) B cells populating the multiple sclerosis brain
mature in the draining cervical lymph nodes. Science translational medicine,
6, 248ra107–248ra107.

Vander Heiden,J.A. et al. (2017) Dysregulation of b cell repertoire
formation in myasthenia gravis patients revealed through deep sequencing.
The Journal of Immunology, 1601415.

Yaari,G. et al. (2012) Quantifying selection in high-throughput
immunoglobulin sequencing data sets. Nucleic acids research, 40,
e134–e134.

Yaari,G. et al. (2013) Models of somatic hypermutation targeting and
substitution based on synonymous mutations from high-throughput
immunoglobulin sequencing data. Frontiers in immunology, 4, 358.
